In [1]:
import uuid
import os
import re
import io
import argparse
import torch
import numpy as np
from PIL import Image

from tool_server.utils.utils import *
from tool_server.utils.server_utils import *
import matplotlib.pyplot as plt

from tool_server.tool_workers.online_workers.base_tool_worker import BaseToolWorker

from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig, BitsAndBytesConfig

2025-04-25 11:19:58 | ERROR | stderr | Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
2025-04-25 11:20:04 | INFO | accelerate.utils.modeling | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
2025-04-25 11:20:39 | ERROR | stderr | /tmp/ipykernel_29427/2944994131.py:2: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
2025-04-25 11:20:39 | ERROR | stderr |   with torch.cuda.amp.autocast(dtype=torch.bfloat16):


In [2]:
# load the processor
model_path = "/mnt/petrelfs/songmingyang/songmingyang/model/mm/tools/Molmo-7B-D-0924"
processor = AutoProcessor.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype='auto',
    device_map='auto'
)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:

text_prompt = "Point to the {} in the scene.".format("Point E")
imput_image_path = "/mnt/petrelfs/songmingyang/code/reasoning/tool-agent/tool_server/tool_workers/online_workers/test_cases/mathvista_35.jpg"
image = Image.open(imput_image_path).convert("RGB")
inputs = processor.process(
    images=[image],
    text=text_prompt,
)
inputs["images"] = inputs["images"].to(torch.bfloat16)
inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

In [4]:
inputs["images"].shape
# inputs.keys()

torch.Size([1, 2, 576, 588])

In [5]:

with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.bfloat16):

        output = model.generate_from_batch(
            inputs,
            GenerationConfig(max_new_tokens=1024, stop_strings="<|endoftext|>"),
            tokenizer=processor.tokenizer
        )

        # only get generated tokens; decode them to text
        generated_tokens = output[0,inputs['input_ids'].size(1):]
        response = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)


In [6]:
response

' <point x="49.8" y="10.0" alt="Point E">Point E</point>'